<a href="https://colab.research.google.com/github/Hysoniya/Emotion_Detection/blob/main/Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets torch sklearn


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install datasets


  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install torch torchvision torchaudio


In [ ]:
pip install transformers scikit-learn


In [ ]:
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, classification_report

# Load the emotion dataset from Hugging Face
dataset = load_dataset('emotion')

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize the text
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenize the datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch tensors
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Split the dataset into train and test sets
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']

# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

# Define a function for evaluation metrics
def compute_metrics(p):
    predictions, labels = p
    preds = torch.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'classification_report': classification_report(labels, preds, target_names=dataset['train'].features['label'].names)
    }

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

# Save the model
model.save_pretrained("./emotion_model")
tokenizer.save_pretrained("./emotion_model")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load the pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)  # Adjust 'num_labels' as per your task
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Set the model to evaluation mode
model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
import torch


In [ ]:
# Import necessary libraries
import torch
from transformers import BertForSequenceClassification, BertTokenizer

# Load the pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)  # Adjust 'num_labels' as per your task
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Set the model to evaluation mode
model.eval()

# Example text for emotion prediction
text = "I am feeling so happy today!"

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt")

# Perform inference (evaluation mode)
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted emotion label (assuming classification)
predictions = torch.argmax(outputs.logits, dim=-1)

# Print the predicted emotion (you can map the prediction to the corresponding emotion)
emotion_labels = ['happy', 'sad', 'angry', 'fearful', 'surprise', 'neutral']  # Modify according to your classes
predicted_emotion = emotion_labels[predictions.item()]
print(f"Predicted Emotion: {predicted_emotion}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted Emotion: angry


In [ ]:
# Import necessary libraries
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load a fine-tuned emotion classification model and tokenizer
model_name = "nateraw/bert-base-uncased-emotion"  # Pre-trained model for emotion detection
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()

# Example text for emotion prediction
text = "I am feeling so happy today!"

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt")

# Perform inference (evaluation mode)
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted emotion label
predictions = torch.argmax(outputs.logits, dim=-1)

# Load emotion labels for this model
emotion_labels = ["sadness", "joy", "love", "anger", "fear", "surprise"]  # Specific to this model
predicted_emotion = emotion_labels[predictions.item()]
print(f"Predicted Emotion: {predicted_emotion}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Predicted Emotion: joy


In [ ]:
pip install transformers torch


In [ ]:
# Import necessary libraries
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Load the pre-trained mBART model and tokenizer
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Set the source and target languages
source_language = "en_XX"  # English
target_language = "hi_IN"  # Hindi

# Update the tokenizer for the target language
tokenizer.src_lang = source_language

# Input text in English
english_text = "Suicide"

# Tokenize the input text
inputs = tokenizer(english_text, return_tensors="pt")

# Generate translation in Hindi
translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[target_language])

# Decode the translated tokens to Hindi text
hindi_translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

# Print the translated text
print(f"English: {english_text}")
print(f"Hindi: {hindi_translation}")


English: Suicide
Hindi: आत्महत्या


In [ ]:
# Import necessary libraries
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Load the pre-trained mBART model and tokenizer
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Set the source and target languages
source_language = "en_XX"  # English
target_language = "hi_IN"  # Hindi

# Update the tokenizer for the target language
tokenizer.src_lang = source_language

# Prompt the user for input text
english_text = input("Enter text in English: ")

# Tokenize the input text
inputs = tokenizer(english_text, return_tensors="pt")

# Generate translation in Hindi
translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[target_language])

# Decode the translated tokens to Hindi text
hindi_translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

# Print the translated text
print(f"\nEnglish: {english_text}")
print(f"Hindi: {hindi_translation}")


Enter text in English: Fig is helpful in curing pneumonoultramicroscopicsilicovolcanoniosis

English: Fig is helpful in curing pneumonoultramicroscopicsilicovolcanoniosis
Hindi: निमोनोउल्ट्रामीक्रोस्कोपिक्सिकोलिकोवोल्कानोयोसिस के उपचार में फाइग सहायक है।
